<a href="https://colab.research.google.com/github/shahzad-r1zv1/LocalLLM_experiments/blob/main/7s.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install -q pandas tqdm matplotlib reportlab python-pptx requests


In [25]:
# ============================== ONE-CELL 7S + OLLAMA (COLAB, WRC EDITION) ==============================
# Installs Ollama, pulls up to 3 models, runs selected 7S prompts across models (WRC wording),
# consolidates results, and saves JSON/Markdown outputs in /content.

import os, subprocess, shlex, time, requests, json, re, textwrap, datetime
from typing import Dict, List

# ---------------------------
# 0) BASIC CONFIG (EDIT THESE)
# ---------------------------
# Organization inputs used to fill prompt placeholders
ORG = {
    "name": "QA Automation Org",
    "industry": "Software / DevTools",
    "size_employees": 180,
    "annual_revenue": "N/A (cost center)",
    "locations": "Toronto; Remote",
    "current_context": "Transitioning from phased QA to DevSecOps; aiming for predictive quality and CI/CD-first culture.",
    "key_challenges": "Inconsistent test coverage; siloed defect data; unclear decision rights; skills gap in pipeline engineering.",
    "recent_changes": "Trunk-based dev in 2 squads; feature flags/canary pilots; Playwright + contract tests on 2 critical APIs.",
    # Optional fields for specific prompts (leave as "" if unknown)
    "strategy_statement": "Ship faster with confidence via automated quality gates and proactive risk analytics.",
    "strategic_goals": "12 months: 70% automated coverage; 24m: 95% critical path; MTTR<1hr; DORA elite.",
    "kpis": "DORA metrics, escaped defect rate, mean time to detect, e2e reliability SLOs.",
    "market_position": "Internal enablement platform within a large enterprise.",
    "competitive_advantages": "Domain expertise; test data virtualization; unified telemetry.",
    "target_segments": "Product teams building microservices/APIs.",
    "value_proposition": "Fewer incidents, faster releases, higher trust in automation.",
    "org_chart": "VP Eng → Dir QA/Platform → QA Enablement, SDET, Observability pods; squads map to product lines.",
    "layers": "VP → Director → Managers → ICs (4 layers)",
    "span_of_control": "Managers: 6-8 ICs avg",
    "decision_rights": "Product owns scope; QA owns quality standards; Dev owns implementation; shared release gates.",
    "xfunc": "Release Council; Reliability Guild; Architecture Forum.",
    "geo": "Toronto hub + distributed remote",
    "restructuring": "Evolving towards platform team model; decoupling test infra from app squads."
}

# Which prompts to run (choose from keys below)
RUN_PROMPTS = [
    "PROMPT_1_FULL_7S",
    "PROMPT_2_STRATEGY",
    "PROMPT_3_STRUCTURE",
    "PROMPT_4_SYSTEMS",
    "PROMPT_5_SHARED_VALUES",
    "PROMPT_6_SKILLS",
    "PROMPT_7_STYLE",
    "PROMPT_8_STAFF",
    "PROMPT_9_ALIGNMENT",
    "PROMPT_10_CHANGE",
    "PROMPT_11_DIGITAL",
    "PROMPT_12_BENCH",
    "PROMPT_13_GAPS",
    "PROMPT_14_INTEGRATION",
    "PROMPT_15_EXEC_SUMMARY",
    "MEGA_PROMPT"
]

In [26]:
# Models to try pulling (we’ll use what actually succeeds), "qwen3:8b"
# REQUESTED_MODELS = ["qwen3", "mistral", "llama3"]
#REQUESTED_MODELS = ["qwen3", "phi3", "llama3"]
# Common alias fixes (Ollama doesn't have "gemma3")
# MODEL_ALIASES = {
#     "gemma3": "gemma2:9b-instruct",
#     "gemma2-9b": "gemma2:9b-instruct",
#     "gemma2-2b": "gemma2:2b-instruct",
#     "gemma7b": "gemma:7b-instruct",
#     "llama3.1": "llama3.1",
#      # pass-through example
# }

# # --- Pull models asynchronously ---
# models = ["mistral", "llama3", "gemma2:2b-instruct"]  # use gemma2:2b or 9b instead of gemma3
# models = ["qwen3", "phi3", "llama3"]

MODEL_ALIASES = {
    "gemma3": "gemma2:9b-instruct",
    "gemma2": "gemma2:2b-instruct",
    "llama3": "llama3",
    "llama3.1": "llama3.1",
    "mistral": "mistral",
    "phi4":"phi4",
    "qwen3" : "qwen3:14b",
    "gemma7b": "gemma:7b-instruct",
}
REQUESTED_MODELS = ["mistral", "llama3", "phi4"]
REQUESTED_MODELS = [MODEL_ALIASES.get(m.lower(), m) for m in REQUESTED_MODELS]

# Output paths
OUT_DIR = "/content"
BUNDLE_JSON = os.path.join(OUT_DIR, "7S_bundle.json")
REPORT_MD   = os.path.join(OUT_DIR, "7S_report.md")

In [27]:
# ===============================================================
# 🚀 OLLAMA INSTALL / START / SMART MODEL PULLER  (with speed + restart)
# ===============================================================
import subprocess, shlex, requests, time, os, sys, re, math

def run(cmd, check=True, quiet=False):
    if isinstance(cmd, str):
        cmd = shlex.split(cmd)
    p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    if not quiet: print(p.stdout.strip())
    if check and p.returncode != 0:
        raise RuntimeError(f"CMD FAIL: {' '.join(cmd)}\n----\n{p.stdout}")
    return p.stdout.strip()

def ollama_installed():
    try:
        out = run("ollama --version", check=False, quiet=True)
        return "ollama" in out.lower()
    except Exception: return False

def ollama_running():
    try:
        r = requests.get("http://127.0.0.1:11434/api/tags", timeout=3)
        return r.status_code == 200
    except Exception: return False

# ------------------- INSTALL -------------------
if ollama_installed():
    print("✅ Ollama already installed.")
else:
    print("📦 Installing Ollama (with progress)...")
    proc = subprocess.Popen(
        "bash -lc 'curl -fSL https://ollama.com/install.sh | sh'",
        shell=True, text=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT
    )
    for line in iter(proc.stdout.readline, ''):
        sys.stdout.write(line); sys.stdout.flush()
    proc.wait()
    print("✅ Installation complete." if proc.returncode == 0 else "⚠️ Installation may have issues!")

# ------------------- START SERVER -------------------
if ollama_running():
    print("✅ Ollama already running.")
else:
    print("🚀 Starting ollama serve…")
    run("bash -lc 'pkill -f \"ollama serve\" || true'", check=False, quiet=True)
    logf = open("/tmp/ollama.log", "w")
    proc = subprocess.Popen(["ollama","serve"], stdout=logf, stderr=subprocess.STDOUT, text=True)
    for _ in range(60):
        if ollama_running(): break
        print(".", end="", flush=True); time.sleep(2)
    print("\n✅ Ollama API ready at http://127.0.0.1:11434")

# ------------------- SMART MODEL PULLER -------------------
def model_exists(m):
    try:
        r = requests.get("http://127.0.0.1:11434/api/tags", timeout=5)
        if r.status_code==200:
            names=[x["name"] for x in r.json().get("models",[])]
            return any(m.startswith(x) or x.startswith(m) for x in names)
    except: return False
    return False

def pull_model(m, max_retries=2, stall_threshold=0.10, stall_time=300):
    """
    Pull model with live progress, MB/s estimate, smart stall recovery,
    and periodic heartbeat for Colab environments.
    """
    MODEL_SIZES = {
        "mistral": 4200, "llama3": 8000, "gemma2:9b-instruct": 9000,
        "phi3": 3800, "gemma2:2b": 2200
    }
    total_mb = MODEL_SIZES.get(m.lower(), 4000)
    attempt = 0

    while attempt <= max_retries:
        attempt += 1
        if model_exists(m):
            print(f"✅ {m} already available.")
            return True

        print(f"\n⬇️ Pulling {m} (attempt {attempt}/{max_retries+1})...")
        p = subprocess.Popen(["ollama", "pull", m],
                             stdout=subprocess.PIPE,
                             stderr=subprocess.STDOUT,
                             text=True, bufsize=1)

        prog = re.compile(r"(\d+)%")
        last_pct, last_time = 0, time.time()
        speed_window = []
        last_update = time.time()
        last_heartbeat = time.time()
        stalled = False

        try:
            for line in iter(p.stdout.readline, ""):
                line = line.strip()
                match = prog.search(line)
                now = time.time()

                if match:
                    pct = int(match.group(1))
                    dt = max(now - last_time, 0.1)
                    dp = pct - last_pct

                    if dp > 0:
                        inst_speed = dp / dt
                        speed_window.append(inst_speed)
                        if len(speed_window) > 5:
                            speed_window.pop(0)
                        avg_speed = sum(speed_window) / len(speed_window)
                        last_time, last_pct = now, pct
                    else:
                        avg_speed = sum(speed_window) / len(speed_window) if speed_window else 0

                    mb_speed = (avg_speed / 100.0) * total_mb / max(dt, 1)
                    eta = min(((100 - pct) / max(avg_speed, 0.01)), 9999)

                    bar = f"[{'='*(pct//5):<20}]"
                    sys.stdout.write(
                        f"\r📦 {m:<18} {bar} {pct:3d}% | {avg_speed:4.2f}%/s | {mb_speed:5.2f} MB/s | ETA {eta:4.0f}s"
                    )
                    sys.stdout.flush()
                    last_update = now

                # Heartbeat if quiet for a while
                elif now - last_heartbeat > 90:
                    sys.stdout.write(" 💓")
                    sys.stdout.flush()
                    last_heartbeat = now

                # Stall detection
                if now - last_update > stall_time:
                    print(f"\n⚠️ {m} stalled (no update for {stall_time}s). Restarting...")
                    p.kill()
                    stalled = True
                    break

            p.wait()
            print()  # newline after progress bar

        except Exception as e:
            print(f"\n⚠️ Error while pulling {m}: {e}")
            p.kill()
            stalled = True

        if not stalled and p.returncode == 0:
            print(f"✅ {m} pull complete.")
            return True
        elif attempt <= max_retries:
            print(f"🔁 Retrying {m} in 10 s...")
            time.sleep(10)
        else:
            print(f"❌ {m} failed (exit {p.returncode}).")
            return False





# ------------------- EXECUTE PULL -------------------
REQUESTED_MODELS = ["mistral", "llama3", "gemma2"]
print("\n📥 Pulling models via running daemon...\n")

for m in REQUESTED_MODELS:
    pull_model(m)

print("\n📋 Models available now:")
!ollama list


✅ Ollama already installed.
🚀 Starting ollama serve…
.
✅ Ollama API ready at http://127.0.0.1:11434

📥 Pulling models via running daemon...

✅ mistral already available.
✅ llama3 already available.
✅ gemma2 already available.

📋 Models available now:
NAME              ID              SIZE      MODIFIED       
mistral:latest    6577803aa9a0    4.4 GB    8 minutes ago     
gemma2:latest     ff02c3702f32    5.4 GB    35 minutes ago    
llama3:latest     365c0bd3c000    4.7 GB    43 minutes ago    


In [28]:

# Base endpoints
BASE = "http://127.0.0.1:11434"
TAGS = f"{BASE}/api/tags"
CHAT = f"{BASE}/api/chat"
GEN  = f"{BASE}/api/generate"
print(f"🌐 Connected to Ollama at {BASE}")
PRIMARY_MODEL = REQUESTED_MODELS[0]

🌐 Connected to Ollama at http://127.0.0.1:11434


In [29]:
# ---------------------------
# 1️⃣ INSTALL & START OLLAMA (idempotent)
# ---------------------------
import subprocess, shlex, requests, time, os

def run(cmd, check=True, quiet=False):
    """Run a shell command and optionally suppress output."""
    if isinstance(cmd, str):
        cmd = shlex.split(cmd)
    p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    if not quiet:
        print(p.stdout.strip())
    if check and p.returncode != 0:
        raise RuntimeError(f"CMD FAIL: {' '.join(cmd)}\n----\n{p.stdout}")
    return p.stdout.strip()

def ollama_installed():
    try:
        out = run("ollama --version", check=False, quiet=True)
        return "ollama" in out.lower() or "version" in out.lower()
    except Exception:
        return False

def ollama_running():
    try:
        r = requests.get("http://127.0.0.1:11434/api/tags", timeout=3)
        return r.status_code == 200
    except Exception:
        return False

# 1. Install Ollama if needed
if ollama_installed():
    print("✅ Ollama already installed.")
else:
    print("📦 Installing Ollama…")
    import subprocess, shlex, sys, time

    # Use non-silent curl (remove -s) so progress bar appears
    install_cmd = "bash -lc 'curl -fSL https://ollama.com/install.sh | sh'"

    # Stream output in real time so you can see download progress
    proc = subprocess.Popen(install_cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

    last_update = time.time()
    for line in iter(proc.stdout.readline, ''):
        if line.strip():
            # throttle slightly so Colab output remains responsive
            if time.time() - last_update > 0.2:
                sys.stdout.write(line)
                sys.stdout.flush()
                last_update = time.time()

    proc.wait()
    if proc.returncode == 0:
        print("✅ Ollama installation complete.")
    else:
        print(f"❌ Installation exited with code {proc.returncode}. Check above logs.")

    run("bash -lc 'curl -fsSL https://ollama.com/install.sh | sh'", check=False, quiet=True)
    print("✅ Installation complete.")

# 2. Start Ollama service (if not already running)
if ollama_running():
    print("✅ Ollama service already running.")
else:
    print("🚀 Starting ollama serve…")
    run("bash -lc 'pkill -f \"ollama serve\" || true'", check=False, quiet=True)
    log_path = "/tmp/ollama.log"
    logf = open(log_path, "w")
    proc = subprocess.Popen(["ollama", "serve"], stdout=logf, stderr=subprocess.STDOUT, text=True)

    print("⏳ Waiting for Ollama to become ready", end="")
    ready = False
    for _ in range(60):
        if ollama_running():
            ready = True
            break
        print(".", end="", flush=True)
        time.sleep(2)
    print()

    if not ready:
        print("⚠️ Ollama did not start. Showing last 30 log lines:")
        try:
            print("".join(open(log_path, "r", errors="ignore").readlines()[-30:]))
        except Exception:
            pass
        raise SystemExit("❌ Cannot continue without Ollama.")
    else:
        print("✅ Ollama API ready at http://127.0.0.1:11434")

# Base endpoints
BASE = "http://127.0.0.1:11434"
TAGS = f"{BASE}/api/tags"
CHAT = f"{BASE}/api/chat"
GEN  = f"{BASE}/api/generate"
print(f"🌐 Connected to Ollama at {BASE}")


✅ Ollama already installed.
✅ Ollama service already running.
🌐 Connected to Ollama at http://127.0.0.1:11434


In [31]:
# ---------------------------
# 3) PROMPT TEMPLATES (WRC wording)
# ---------------------------
def fill(s: str, kv: Dict[str,str]) -> str:
    # simple replacement of [FIELD] with kv.get(field_lower)
    def rep(m):
        key = m.group(1).strip().lower().replace(" ", "_")
        return str(kv.get(key, f"N/A: {m.group(0)}"))
    return re.sub(r"\[([^\]]+)\]", rep, s)

PROMPTS = {
"PROMPT_1_FULL_7S": """Act as a WRC (World Renowned Consultancy) consultant conducting a comprehensive 7S analysis.
INPUTS NEEDED:
- Company/Organization name: [NAME]
- Industry: [INDUSTRY]
- Company size (employees): [NUMBER]
- Annual revenue: [REVENUE]
- Geographic presence: [LOCATIONS]
- Current business context: [BRIEF DESCRIPTION]
- Key challenges facing the organization: [LIST 3-5]
- Recent major changes or initiatives: [DESCRIBE]
Analyze all seven elements:
1. Strategy (competitive positioning, strategic priorities, value proposition)
2. Structure (org design, reporting lines, decision rights)
3. Systems (processes, IT systems, workflows)
4. Shared Values (culture, core beliefs, mission/vision)
5. Style (leadership approach, management practices)
6. Staff (workforce composition, capabilities, engagement)
7. Skills (organizational competencies, competitive advantages)
OUTPUT FORMAT:
- Executive summary with key findings
- Detailed analysis of each S (current state assessment)
- Interconnections and dependencies between elements
- Alignment score (1-10) for each element pair
- Top 5 misalignments requiring attention
- Recommendations prioritized by impact and feasibility
""",
"PROMPT_2_STRATEGY": """Evaluate the clarity and effectiveness of our organizational strategy using the 7S model.
INPUTS NEEDED:
- Current strategy statement: [PROVIDE]
- Strategic goals (3-5 year): [LIST]
- Key performance indicators: [LIST]
- Market position: [DESCRIBE]
- Competitive advantages: [LIST]
- Target customer segments: [DESCRIBE]
- Value proposition: [STATEMENT]
Assess our strategy by examining:
1. Clarity and communication throughout organization
2. Alignment with market opportunities
3. Differentiation from competitors
4. Resource allocation alignment
5. Measurability and tracking mechanisms
6. Connection to other 6 S elements
OUTPUT:
- Strategy effectiveness score (1-10) with justification
- Strengths and weaknesses analysis
- Gaps between stated and actual strategy
- Impact assessment on other 6 S elements
- 5 specific recommendations to strengthen strategy
- Communication plan to improve strategy understanding
""",
"PROMPT_3_STRUCTURE": """Analyze our organizational structure's alignment with strategic goals.
INPUTS NEEDED:
- Current org chart: [DESCRIBE HIERARCHY]
- Number of management layers: [NUMBER]
- Span of control averages: [NUMBERS]
- Decision-making authority levels: [DESCRIBE]
- Cross-functional teams/committees: [LIST]
- Geographic/divisional structure: [DESCRIBE]
- Recent restructuring efforts: [DESCRIBE IF ANY]
Evaluate:
1. Structure-strategy fit
2. Decision-making speed and effectiveness
3. Communication flow efficiency
4. Collaboration barriers
5. Duplication or gaps in responsibilities
6. Flexibility for future growth
OUTPUT:
- Structure effectiveness rating with evidence
- Organizational design recommendations
- Proposed org chart modifications
- Impact analysis on systems and staff
- Implementation roadmap for structural changes
- Risk assessment of proposed changes
""",
"PROMPT_4_SYSTEMS": """Assess the effectiveness of operational and management systems.
INPUTS NEEDED:
- Core business processes: [LIST]
- IT systems and platforms: [LIST]
- Performance management systems: [DESCRIBE]
- Financial/budgeting processes: [DESCRIBE]
- Quality control systems: [DESCRIBE]
- Communication systems: [DESCRIBE]
- Decision-making processes: [DESCRIBE]
- Knowledge management systems: [DESCRIBE]
Analyze:
1. Process efficiency and effectiveness
2. System integration and data flow
3. Automation opportunities
4. Performance tracking capabilities
5. User satisfaction and adoption
6. Alignment with strategic objectives
OUTPUT:
- Systems maturity assessment (1-5 scale per system)
- Critical system gaps and redundancies
- Process optimization opportunities
- Technology upgrade recommendations
- Implementation priority matrix
- ROI estimates for system improvements
""",
"PROMPT_5_SHARED_VALUES": """Identify and evaluate core shared values driving culture and decision-making.
INPUTS NEEDED:
- Stated mission/vision/values: [PROVIDE]
- Employee survey results: [KEY FINDINGS]
- Leadership behaviors observed: [EXAMPLES]
- Decision-making patterns: [DESCRIBE]
- Customer feedback themes: [SUMMARIZE]
- Internal communication samples: [PROVIDE EXAMPLES]
- Recognition and reward criteria: [LIST]
Examine:
1. Stated vs. lived values gap analysis
2. Values alignment across hierarchy
3. Values impact on behaviors
4. Cultural strengths and toxicities
5. Values-strategy alignment
6. Employee value perception
OUTPUT:
- Core values identification (top 5 actual vs. stated)
- Cultural health score with supporting evidence
- Values-behavior alignment matrix
- Cultural transformation requirements
- Values reinforcement action plan
- Measurement framework for cultural change
""",
"PROMPT_6_SKILLS": """Evaluate critical skills and competencies across the organization.
INPUTS NEEDED:
- Current workforce skills inventory: [CATEGORIES]
- Strategic capability requirements: [LIST]
- Competitor capabilities: [BENCHMARK DATA]
- Training and development programs: [DESCRIBE]
- Performance review data: [KEY METRICS]
- Skills gaps identified by managers: [LIST]
- Future skill requirements (3-5 years): [ANTICIPATE]
Assess:
1. Current vs. required skills gaps
2. Core competency strengths
3. Competitive skill advantages/disadvantages
4. Skills development effectiveness
5. Knowledge transfer mechanisms
6. Innovation and adaptation capabilities
OUTPUT:
- Skills heat map (current vs. required)
- Critical skills gap analysis with risk levels
- Competency development roadmap
- Make/buy/partner talent decisions
- L&D investment recommendations
- Skills KPIs and tracking mechanisms
""",
"PROMPT_7_STYLE": """Analyze the dominant leadership style and its impact on performance.
INPUTS NEEDED:
- Leadership team composition: [DESCRIBE]
- Leadership assessment data: [IF AVAILABLE]
- Employee engagement scores: [PROVIDE]
- Decision-making examples: [3-5 CASES]
- Communication patterns: [DESCRIBE]
- Change management approaches: [EXAMPLES]
- Succession planning status: [DESCRIBE]
Evaluate:
1. Predominant leadership styles
2. Leadership effectiveness metrics
3. Style-strategy alignment
4. Leadership impact on culture
5. Decision-making patterns
6. Leadership development needs
OUTPUT:
- Leadership style profile with strengths/weaknesses
- Leadership effectiveness score (1-10)
- Style-situation fit analysis
- Leadership development priorities
- Succession planning recommendations
- Leadership behavior change roadmap
""",
"PROMPT_8_STAFF": """Review workforce composition, recruitment, and retention strategies.
INPUTS NEEDED:
- Total headcount and demographics: [PROVIDE]
- Organizational structure by function: [BREAKDOWN]
- Turnover rates by level/function: [DATA]
- Time-to-fill metrics: [AVERAGES]
- Employee engagement scores: [PROVIDE]
- Compensation benchmarking: [POSITION VS MARKET]
- Talent pipeline status: [DESCRIBE]
- Diversity metrics: [PROVIDE]
Analyze:
1. Workforce composition vs. strategic needs
2. Talent acquisition effectiveness
3. Retention risks and drivers
4. Engagement and productivity levels
5. Diversity, equity, and inclusion status
6. Workforce planning adequacy
OUTPUT:
- Workforce health scorecard
- Critical talent risks and mitigation plans
- Recruitment strategy optimization
- Retention program enhancements
- Workforce planning recommendations
- HR metrics dashboard design
""",
"PROMPT_9_ALIGNMENT": """Evaluate how well all seven elements of the 7S Framework align.
INPUTS NEEDED:
- Brief assessment of each S element: [PROVIDE STATUS]
- Recent organizational changes: [LIST]
- Performance metrics trends: [LAST 2 YEARS]
- Strategic priorities: [TOP 5]
- Known pain points: [DESCRIBE]
- Success stories: [EXAMPLES]
Assess:
1. Element interdependencies and conflicts
2. Alignment scoring for each element pair (21 combinations)
3. Reinforcing vs. conflicting relationships
4. Impact of misalignments on performance
5. Root cause analysis of gaps
6. Synergy opportunities
OUTPUT:
- 7S alignment matrix with scores
- Critical misalignment identification
- Dependency map visualization
- Prioritized realignment initiatives
- Change sequencing recommendations
- Alignment monitoring framework
""",
"PROMPT_10_CHANGE": """Use the 7S Framework to analyze organizational readiness for change.
INPUTS NEEDED:
- Planned change initiative: [DESCRIBE]
- Change timeline and scope: [PROVIDE]
- Previous change efforts: [SUCCESS/FAILURE EXAMPLES]
- Stakeholder groups affected: [LIST]
- Current change capability maturity: [ASSESS 1-5]
- Resource availability: [BUDGET/PEOPLE]
- Risk tolerance: [LOW/MEDIUM/HIGH]
Analyze each S element for:
1. Current state change readiness
2. Required changes per element
3. Resistance points and drivers
4. Change capability gaps
5. Success enablers and barriers
6. Change impact assessment
OUTPUT:
- Change readiness score by S element
- Resistance heat map
- Change impact assessment matrix
- Stakeholder engagement strategy
- Change roadmap with quick wins
- Risk mitigation plan
- Success metrics framework
""",
"PROMPT_11_DIGITAL": """Apply the 7S Framework to assess digital transformation readiness and impact.
INPUTS NEEDED:
- Current digital maturity: [ASSESS 1-5]
- Digital strategy/initiatives: [DESCRIBE]
- Technology infrastructure: [CURRENT STATE]
- Digital skills inventory: [ASSESS]
- Data and analytics capabilities: [DESCRIBE]
- Customer digital expectations: [SUMMARIZE]
- Competitor digital positioning: [BENCHMARK]
Evaluate digital impact on:
1. Strategy (digital business models)
2. Structure (agile organization needs)
3. Systems (technology architecture)
4. Shared Values (digital culture)
5. Style (digital leadership)
6. Staff (digital talent)
7. Skills (digital capabilities)
OUTPUT:
- Digital maturity assessment by S element
- Digital transformation gaps and priorities
- Technology investment recommendations
- Digital culture transformation plan
- Reskilling/upskilling requirements
- Digital KPIs and governance model
- Transformation roadmap with milestones
""",
"PROMPT_12_BENCH": """Compare our 7S profile against key competitors.
INPUTS NEEDED:
- Top 3-5 competitors: [LIST]
- Competitive intelligence available: [SUMMARIZE]
- Industry best practices: [DESCRIBE]
- Our performance vs. competitors: [METRICS]
- Competitive advantages/disadvantages: [LIST]
- Market position: [DESCRIBE]
Benchmark:
1. Strategy effectiveness comparison
2. Organizational agility assessment
3. Operational excellence comparison
4. Cultural differentiation analysis
5. Leadership capability comparison
6. Talent competitiveness evaluation
7. Innovation capability assessment
OUTPUT:
- Competitive 7S comparison matrix
- Competitive advantage/disadvantage analysis
- Best practice identification
- Competitive gaps requiring closure
- Differentiation opportunities
- Competitive response strategies
- Monitoring and intelligence framework
""",
"PROMPT_13_GAPS": """Identify gaps between current and desired future state across the 7S elements.
INPUTS NEEDED:
- Vision for future state (3-5 years): [DESCRIBE]
- Current state assessment: [SUMMARIZE BY S]
- Strategic objectives: [LIST]
- Performance targets: [SPECIFY]
- Market/industry trends: [IDENTIFY]
- Stakeholder expectations: [DESCRIBE]
Analyze:
1. Current state baseline for each S
2. Future state requirements per S
3. Gap magnitude and complexity
4. Interdependency impact analysis
5. Resource requirements for gap closure
6. Timeline and sequencing needs
OUTPUT:
- Current vs. future state comparison table
- Gap severity assessment (critical/high/medium/low)
- Gap closure difficulty matrix
- Investment requirements estimate
- Transformation roadmap with phases
- Quick wins vs. long-term initiatives
- Success metrics and milestones
""",
"PROMPT_14_INTEGRATION": """Combine 7S insights with SWOT or PESTLE analysis.
INPUTS NEEDED:
- 7S assessment summary: [PROVIDE]
- SWOT analysis: [IF AVAILABLE]
- PESTLE factors: [IF AVAILABLE]
- Strategic options under consideration: [LIST]
- Risk factors identified: [LIST]
- Opportunity areas: [DESCRIBE]
Integrate analyses to:
1. Map external factors to internal capabilities
2. Identify strategic option feasibility
3. Assess implementation capabilities
4. Determine competitive positioning
5. Evaluate risk mitigation capacity
6. Prioritize strategic initiatives
OUTPUT:
- Integrated strategy framework
- Strategic option evaluation matrix
- Capability-opportunity alignment map
- Risk-readiness assessment
- Strategic initiative prioritization
- Implementation feasibility scores
- Integrated dashboard design
""",
"PROMPT_15_EXEC_SUMMARY": """Create an executive-level summary of 7S analysis with actionable recommendations.
INPUTS NEEDED:
- Full 7S analysis results: [PROVIDE KEY FINDINGS]
- Strategic context and urgency: [DESCRIBE]
- Available resources: [BUDGET/CAPACITY]
- Board/Executive priorities: [LIST]
- Key stakeholder concerns: [IDENTIFY]
- Success criteria: [DEFINE]
Synthesize:
1. Critical insights from 7S analysis
2. Top 3-5 strategic imperatives
3. Quick wins vs. transformational changes
4. Investment requirements and ROI
5. Risk assessment and mitigation
6. Implementation timeline
OUTPUT:
- 2-page executive summary
- Visual 7S alignment dashboard
- Top 10 recommendations ranked by impact
- Investment and resource requirements
- 90-day, 6-month, and 1-year action plans
- Success metrics and governance model
- Key risks and mitigation strategies
- Next steps and decision requirements
""",
"MEGA_PROMPT": """Act as a senior WRC consultant conducting a comprehensive 7S analysis. Provide a complete organizational assessment with actionable insights.
[Use all context provided above and infer reasonable assumptions where N/A.]
DELIVERABLES:
- Executive Summary; Detailed Analysis by S; Integrated Findings; Strategic Recs; Implementation Plan; Appendices.
"""
}

# Map bracketed placeholders to ORG fields
PLACEHOLDER_MAP = {
    "[NAME]": ORG.get("name",""),
    "[INDUSTRY]": ORG.get("industry",""),
    "[NUMBER]": ORG.get("size_employees",""),
    "[REVENUE]": ORG.get("annual_revenue",""),
    "[LOCATIONS]": ORG.get("locations",""),
    "[BRIEF DESCRIPTION]": ORG.get("current_context",""),
    "[LIST 3-5]": ORG.get("key_challenges",""),
    "[DESCRIBE]": ORG.get("recent_changes",""),
    "[PROVIDE]": ORG.get("strategy_statement",""),
    "[LIST]": ORG.get("strategic_goals",""),
    "[STATEMENT]": ORG.get("value_proposition",""),
    "[DESCRIBE HIERARCHY]": ORG.get("org_chart",""),
    "[NUMBERS]": ORG.get("span_of_control",""),
    "[IF AVAILABLE]": "",
    "[AVERAGES]": "",
    "[BREAKDOWN]": "",
    "[DATA]": "",
    "[POSITION VS MARKET]": "",
    "[ASSESS 1-5]": "",
    "[CURRENT STATE]": "",
    "[ASSESS]": "",
    "[SUMMARIZE]": "",
    "[ANALYZE]": "",
    "[IDENTIFY]": "",
    "[SPECIFY]": ""
}

def materialize_prompt(key: str) -> str:
    text = PROMPTS[key]
    for k,v in PLACEHOLDER_MAP.items():
        text = text.replace(k, str(v if v else f"N/A: {k}"))
    return text


In [32]:
# ===============================================================
# 🧠 OLLAMA CHAT HELPERS — Auto-detects generate/chat endpoint
# ===============================================================
import requests, json, time
from typing import Dict

TEMPERATURE, TOP_P, REPEAT_PENALTY, TIMEOUT_S = 0.4, 0.9, 1.1, 860

# ---------------------------------------------------------------
# 1️⃣ Detect endpoint availability and set USE_GENERATE
# ---------------------------------------------------------------
BASE = "http://127.0.0.1:11434"
GEN  = f"{BASE}/api/generate"
CHAT = f"{BASE}/api/chat"

def detect_generate_support():
    try:
        r = requests.options(GEN, timeout=3)
        if r.status_code in (200, 204):
            return True
    except Exception:
        pass
    return False

USE_GENERATE = detect_generate_support()
print(f"⚙️ Using {'/api/generate' if USE_GENERATE else '/api/chat'} endpoint for inference.")

# ---------------------------------------------------------------
# 2️⃣ Ask LLM — general wrapper
# ---------------------------------------------------------------
def ask_llm(model: str, sys: str, user: str) -> str:
    """Send message to Ollama and return text output."""
    try:
        if USE_GENERATE:
            # Simpler, faster API (no chat format)
            prompt = f"<<SYS>>\n{sys}\n<</SYS>>\n\n{user}"
            r = requests.post(
                GEN,
                json={
                    "model": model,
                    "prompt": prompt,
                    "stream": False,
                    "options": {
                        "temperature": TEMPERATURE,
                        "top_p": TOP_P,
                        "repeat_penalty": REPEAT_PENALTY
                    },
                },
                timeout=TIMEOUT_S,
            )
            r.raise_for_status()
            return (r.json().get("response") or "").strip()
        else:
            # Chat-style API
            r = requests.post(
                CHAT,
                json={
                    "model": model,
                    "messages": [
                        {"role": "system", "content": sys},
                        {"role": "user", "content": user},
                    ],
                    "stream": False,
                    "options": {
                        "temperature": TEMPERATURE,
                        "top_p": TOP_P,
                        "repeat_penalty": REPEAT_PENALTY,
                    },
                },
                timeout=TIMEOUT_S,
            )
            r.raise_for_status()
            return ((r.json().get("message") or {}).get("content") or "").strip()

    except requests.exceptions.Timeout:
        return "⚠️ Request timed out — model took too long."
    except Exception as e:
        return f"❌ Error querying model {model}: {e}"

# ---------------------------------------------------------------
# 3️⃣ Multi-model consensus synthesis
# ---------------------------------------------------------------
def synthesize_consensus(primary_model: str, prompt_name: str, per_model: Dict[str, str]) -> str:
    """Combine responses from multiple models into one cohesive expert synthesis."""
    sys = (
        "You are a senior WRC consultant. "
        "Synthesize multiple expert drafts into one coherent, non-redundant, executive-ready report."
    )
    user = f"""Prompt: {prompt_name}

Combine the following {len(per_model)} model answers into one structured response.
Do NOT mention that this is a synthesis. Present it like a polished consultant report.

Guidelines:
- Keep the best points from each.
- Merge ideas logically.
- If answers conflict, show trade-offs and state your recommendation.
- Use concise sections and bulleted clarity.

=== MODEL ANSWERS ===
{json.dumps(per_model, indent=2, ensure_ascii=False)}
"""
    return ask_llm(primary_model, sys, user)


⚙️ Using /api/chat endpoint for inference.


In [ ]:
# ---------------------------
# 5) RUN SELECTED PROMPTS ACROSS MODELS
# ---------------------------
def run_prompt_across_models(prompt_key: str, models: List[str]) -> Dict[str,str]:
    sys = "You are precise and to-the-point."
    usr = materialize_prompt(prompt_key)
    out = {}
    for m in models:
        try:
            txt = ask_llm(m, sys, usr)
            out[m] = txt
            print(f"  ✓ {prompt_key} via {m}: {len(txt)} chars")
        except Exception as e:
            out[m] = f"(error from {m}: {e})"
            print(f"  ✗ {prompt_key} via {m}: {e}")
    return out

results = {}
consensus = {}
print("\n🧠 Running prompts:", ", ".join(RUN_PROMPTS))
for key in RUN_PROMPTS:
    per_model = run_prompt_across_models(key, REQUESTED_MODELS)
    results[key] = per_model
    consensus[key] = synthesize_consensus(PRIMARY_MODEL, key, per_model)





🧠 Running prompts: PROMPT_1_FULL_7S, PROMPT_2_STRATEGY, PROMPT_3_STRUCTURE, PROMPT_4_SYSTEMS, PROMPT_5_SHARED_VALUES, PROMPT_6_SKILLS, PROMPT_7_STYLE, PROMPT_8_STAFF, PROMPT_9_ALIGNMENT, PROMPT_10_CHANGE, PROMPT_11_DIGITAL, PROMPT_12_BENCH, PROMPT_13_GAPS, PROMPT_14_INTEGRATION, PROMPT_15_EXEC_SUMMARY, MEGA_PROMPT
  ✓ PROMPT_1_FULL_7S via mistral: 4282 chars
  ✓ PROMPT_1_FULL_7S via llama3: 4601 chars
  ✓ PROMPT_1_FULL_7S via gemma2: 5447 chars
  ✓ PROMPT_2_STRATEGY via mistral: 3168 chars
  ✓ PROMPT_2_STRATEGY via llama3: 3464 chars
  ✓ PROMPT_2_STRATEGY via gemma2: 4283 chars
  ✓ PROMPT_3_STRUCTURE via mistral: 3825 chars
  ✓ PROMPT_3_STRUCTURE via llama3: 4015 chars
  ✓ PROMPT_3_STRUCTURE via gemma2: 5498 chars
  ✓ PROMPT_4_SYSTEMS via mistral: 4052 chars
  ✓ PROMPT_4_SYSTEMS via llama3: 3581 chars
  ✓ PROMPT_4_SYSTEMS via gemma2: 6593 chars
  ✓ PROMPT_5_SHARED_VALUES via mistral: 3797 chars
  ✓ PROMPT_5_SHARED_VALUES via llama3: 4096 chars
  ✓ PROMPT_5_SHARED_VALUES via gemma2: 10

In [ ]:
# ---------------------------
# 6) SAVE OUTPUTS
# ---------------------------
bundle = {
    "metadata": {
        "generated_at_utc": datetime.datetime.now(datetime.timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ"),
        "ollama_host": BASE,
        "models": available,
        "primary_model": PRIMARY_MODEL
    },
    "org_inputs": ORG,
    "raw_per_prompt": results,
    "consensus_per_prompt": consensus
}

with open(BUNDLE_JSON, "w", encoding="utf-8") as f:
    json.dump(bundle, f, indent=2, ensure_ascii=False)

with open(REPORT_MD, "w", encoding="utf-8") as f:
    f.write(f"# 7S Report — Consensus (models: {', '.join(available)})\n\n")
    f.write("## Organization\n")
    f.write("```json\n" + json.dumps(ORG, indent=2, ensure_ascii=False) + "\n```\n\n")
    for key in RUN_PROMPTS:
        f.write(f"## {key}\n\n")
        f.write(consensus.get(key,"").strip() + "\n\n")
        f.write("<details><summary>Model answers</summary>\n\n")
        f.write("```json\n" + json.dumps(results.get(key,{}), indent=2, ensure_ascii=False) + "\n```\n")
        f.write("</details>\n\n")

print("\n✅ Done.")
print("JSON:", BUNDLE_JSON)
print("MD  :", REPORT_MD)
# ==========================================================================================

In [ ]:
# ======================= DISPLAY 7S REPORT IN-CELL =======================
import os, json, textwrap
from IPython.display import display, Markdown, HTML

OUT_DIR = "/content"
BUNDLE_JSON = os.path.join(OUT_DIR, "7S_bundle.json")
REPORT_MD   = os.path.join(OUT_DIR, "7S_report.md")

def _h2(s):
    return f"\n## {s}\n"
def _h3(s):
    return f"\n### {s}\n"
def _codeblock(label, obj):
    if isinstance(obj, str):
        body = obj
    else:
        body = json.dumps(obj, indent=2, ensure_ascii=False)
    return f"\n**{label}**\n\n```json\n{body}\n```\n"

# 1) If the Markdown report exists, render that directly (richest output)
if os.path.exists(REPORT_MD):
    with open(REPORT_MD, "r", encoding="utf-8") as f:
        md = f.read()
    display(Markdown(md))
else:
    # 2) Otherwise, pretty-print from the JSON bundle
    if not os.path.exists(BUNDLE_JSON):
        raise FileNotFoundError("No report found. Run the main cell first to create /content/7S_bundle.json or /content/7S_report.md.")

    with open(BUNDLE_JSON, "r", encoding="utf-8") as f:
        bundle = json.load(f)

    meta     = bundle.get("metadata", {})
    org      = bundle.get("org_inputs", {})
    consensus = bundle.get("consensus_per_prompt", {})
    raw       = bundle.get("raw_per_prompt", {})

    # Build a friendly Markdown view
    parts = []
    title = f"# 7S Report — Consensus (models: {', '.join(meta.get('models', []))})"
    parts.append(title)
    parts.append(_h2("Organization"))
    parts.append("```json\n" + json.dumps(org, indent=2, ensure_ascii=False) + "\n```")

    # Show consensus outputs section-by-section
    for k in consensus.keys():
        parts.append(_h2(k))
        txt = consensus.get(k, "").strip()
        if not txt:
            parts.append("_No consensus text produced for this prompt._")
        else:
            # Light formatting: ensure lines not too long for the notebook width
            wrapped = "\n".join(textwrap.fill(line, width=100) for line in txt.splitlines())
            parts.append(wrapped)

        # Collapsible raw model answers
        parts.append("\n<details><summary>Model answers</summary>\n\n")
        parts.append("```json\n" + json.dumps(raw.get(k, {}), indent=2, ensure_ascii=False) + "\n```")
        parts.append("\n</details>\n")

    display(Markdown("\n".join(parts)))
# ========================================================================
